# Overlapping analysis

In [5]:
import os
import matplotlib.pyplot as plt
from IPython.display import display, Markdown
import pandas as pd

from snl_stats_extraction_data import *
from snl_stats_visualization_database import *
DIR, databases_pair_paths, databases_paths, tier_lists, databases, databases_pairs, tiers = get_parameters()

### Parameters

In [6]:
databases_name = [key.replace('_paths','').upper() for key in databases.keys()]
databases_pairs = [key for key in databases_pairs.keys()]
expressions = ["Smiles_0", "Laughs_0"]
# entities = {expression : tier_lists[expression] for expression in expressions}
laughs_intensities = tier_lists['Laughs_0']
smiles_intensities = tier_lists['Smiles_0']

## Pourcentage of overlap

For each pair of file, we compute the percentage of overlap between the two person in the interaction. 

Here we watch the pourcentage of overlap for each pair of file for list A to list B. (The total duration come from the files of list A)

In [10]:
lstA = {}
lstB = {}
overlapping_segments_dict = {}
for i, database in enumerate(databases_name):
    if database == databases_pairs[i].replace('_pairs', '').upper():
        databases_list = databases_pair_paths[databases_pairs[i]]
        dataset_dict = {}
        for i in range(0, len(databases_list), 2):
            filepath_A = databases_list[i]
            filepath_B = databases_list[i+1]
            pair_name = f"Pair {i//2 + 1}"  
            pair_file_A = os.path.basename(filepath_A)
            pair_file_B = os.path.basename(filepath_B)

            if pair_file_A and pair_file_B:
                pair_name = f"{pair_file_A}_&_{pair_file_B}"

            pair_dict = {}
            for tier in expressions + ["Role"]:
                lstA_tier = get_tier_from_file(filepath_A, tier)
                lstB_tier = get_tier_from_file(filepath_B, tier)

                if tier in lstA:
                    lstA[tier].extend(lstA_tier[tier])
                else:
                    lstA[tier] = lstA_tier[tier]

                if tier in lstB:
                    lstB[tier].extend(lstB_tier[tier])
                else:
                    lstB[tier] = lstB_tier[tier]

                overlapping_segments = get_overlapping_segments(lstA_tier[tier], lstB_tier[tier])
                pair_dict[tier] = {'Segments': overlapping_segments}

            dataset_dict[pair_name] = pair_dict

        overlapping_segments_dict[database] = dataset_dict

dataframes = {}  
for database, dataset_dict in overlapping_segments_dict.items():
    data = [] 
    for pair_name, pair_dict in dataset_dict.items():
        percentages = {}
        for tier, tier_dict in pair_dict.items():
            segments = tier_dict['Segments']
            if not segments:
                percentage = 0 
            else:
                overlapping_duration = sum(segment[1] - segment[0] for segment in segments)
                total_duration = sum(max(segment[1] for segment in segments) for segment in lstA[tier])
                percentage = (overlapping_duration / total_duration) * 100
            percentages[tier] = percentage
        
        data.append({'Pairs filenames': pair_name, **percentages})

    df = pd.DataFrame(data)
    df = df.applymap(lambda x: f"{x:.5f}%" if isinstance(x, float) else x)
    dataframes[database] = df

for database, df in dataframes.items():
    display(Markdown(f"**Dataset: {database}**"))
    display(df)

**Dataset: CCDB**

,Pairs filenames,Smiles_0,Laughs_0,Role
0,P12_P2_1402_dizzy.eaf_&_P12_P2_1402_monk.eaf,0.09024%,0.01519%,0.20647%
1,P12_P3_2202_dizzy.eaf_&_P12_P3_2202_monk.eaf,0.09368%,0.00000%,0.20603%
2,P14_P7_2502_dizzy.eaf_&_P14_P7_2502_monk.eaf,0.06412%,0.00000%,0.21138%
3,P15_P13_2402_dizzy.eaf_&_P15_P13_2402_monk.eaf,0.04787%,0.01374%,0.20972%
4,P16_P6_0903_dizzy.eaf_&_P16_P6_0903_monk.eaf,0.06698%,0.00000%,0.20355%
5,P18_P10_2102_dizzy.eaf_&_P18_P10_2102_monk.eaf,0.07456%,0.00000%,0.20571%
6,P18_P1_2102_dizzy.eaf_&_P18_P1_2102_monk.eaf,0.09983%,0.00000%,0.20214%


**Dataset: IFADV**

,Pairs filenames,Smiles_0,Laughs_0,Role
0,DVA1A.eaf_&_DVB1B.eaf,0.04593%,0.00000%,0.21091%
1,DVA2C.eaf_&_DVB2D.eaf,0.07765%,0.00000%,0.21137%
2,DVA3E.eaf_&_DVB3F.eaf,0.05090%,0.07258%,0.21124%
3,DVA4C.eaf_&_DVB4G.eaf,0.09189%,0.00000%,0.21138%
4,DVA5G.eaf_&_DVB5H.eaf,0.03088%,0.00000%,0.21132%
5,DVA6H.eaf_&_DVB6I.eaf,0.04178%,0.00000%,0.21135%
6,DVA7B.eaf_&_DVB7J.eaf,0.09488%,0.00000%,0.21126%
7,DVA8K.eaf_&_DVB8L.eaf,0.01129%,0.37210%,0.21135%


**Dataset: NDC**

,Pairs filenames,Smiles_0,Laughs_0,Role
0,13_1_A_M.eaf_&_13_1_B_F.eaf,0.02365%,0.02838%,0.21139%
1,13_2_A_M.eaf_&_13_2_B_F.eaf,0.04577%,0.00000%,0.20782%
2,13_4_A_M.eaf_&_13_4_B_F.eaf,0.02932%,0.01518%,0.21139%
3,14_1_A_M.eaf_&_14_1_B_F.eaf,0.00770%,0.00000%,0.17502%
4,14_2_A_M.eaf_&_14_2_B_F.eaf,0.01849%,0.03522%,0.20650%
5,17_1_A_F.eaf_&_17_1_B_F.eaf,0.03408%,0.05452%,0.20226%
6,17_2_A_F.eaf_&_17_2_B_F.eaf,0.05788%,0.03958%,0.20708%
7,17_3_A_F.eaf_&_17_3_B_F.eaf,0.01926%,0.05133%,0.20575%
8,17_4_A_F.eaf_&_17_4_B_F.eaf,0.06149%,0.01993%,0.19157%
9,18_1_A_M.eaf_&_18_1_B_M.eaf,0.04906%,0.04191%,0.21137%


For a more datailed analysis, we show here the duration of overlap for each pair of file for list A to list B, the total duration of the tier of list A and the pourcentage of overlap between A and B.

In [15]:
lstA = {}
lstB = {}
overlapping_segments_dict = {}
for i, database in enumerate(databases_name):
    if database == databases_pairs[i].replace('_pairs', '').upper():
        databases_list = databases_pair_paths[databases_pairs[i]]
        dataset_dict = {}
        for i in range(0, len(databases_list), 2):
            filepath_A = databases_list[i]
            filepath_B = databases_list[i+1] 
            pair_file_A = os.path.basename(filepath_A)
            pair_file_B = os.path.basename(filepath_B)

            if pair_file_A and pair_file_B:
                pair_name = f"{pair_file_A}_&_{pair_file_B}"

            pair_dict = {}
            for tier in expressions + ["Role"]:
                lstA_tier = get_tier_from_file(filepath_A, tier)
                lstB_tier = get_tier_from_file(filepath_B, tier)

                if tier in lstA:
                    lstA[tier].extend(lstA_tier[tier])
                else:
                    lstA[tier] = lstA_tier[tier]

                if tier in lstB:
                    lstB[tier].extend(lstB_tier[tier])
                else:
                    lstB[tier] = lstB_tier[tier]

                overlapping_segments = get_overlapping_segments(lstA_tier[tier], lstB_tier[tier])
                pair_dict[tier] = {'Segments': overlapping_segments}

            dataset_dict[pair_name] = pair_dict

        overlapping_segments_dict[database] = dataset_dict

dataframes = {}
for database, dataset_dict in overlapping_segments_dict.items():
    tiers = expressions + ["Role"]
    data = {tier: [] for tier in tiers}
    for pair_name, pair_dict in dataset_dict.items():
        for tier in tiers:
            segments = pair_dict[tier]['Segments']
            if not segments:
                overlap_duration = 0
                total_duration = 0
                percentage = 0
            else:
                overlap_duration = sum(segment[1] - segment[0] for segment in segments)
                total_duration = sum(max(segment[1] for segment in segments) for segment in lstA[tier])
                percentage = (overlap_duration / total_duration) * 100
            data[tier].append({
                'Pairs filenames': pair_name,
                f'Overlap Percentage for {tier} (%)': percentage,
                f'Total Tier Duration for {tier} (ms)': total_duration,
                f'Overlap Duration for {tier} (ms)': overlap_duration
            })

    dfs = []
    for tier in tiers:
        df = pd.DataFrame(data[tier])
        dfs.append(df)

    df_merged = pd.concat(dfs, axis=1)
    df_merged = df_merged.loc[:, ~df_merged.columns.duplicated()]

    # Calculer le total de chaque colonne
    df_total = df_merged.sum(numeric_only=True)
    df_total['Pairs filenames'] = 'Total'

    # Ajouter la ligne "Total" à la fin du DataFrame
    df_merged = pd.concat([df_merged, pd.DataFrame(df_total).T], ignore_index=True)

    dataframes[database] = df_merged

for database, df in dataframes.items():
    display(Markdown(f"**Database: {database}**"))
    display(df)

**Database: CCDB**

,Pairs filenames,Overlap Percentage for Smiles_0 (%),Total Tier Duration for Smiles_0 (ms),Overlap Duration for Smiles_0 (ms),Overlap Percentage for Laughs_0 (%),Total Tier Duration for Laughs_0 (ms),Overlap Duration for Laughs_0 (ms),Overlap Percentage for Role (%),Total Tier Duration for Role (ms),Overlap Duration for Role (ms)
0,P12_P2_1402_dizzy.eaf_&_P12_P2_1402_monk.eaf,0.090245,118006940,106495,0.015187,8659000,1315,0.206472,56835680,117350
1,P12_P3_2202_dizzy.eaf_&_P12_P3_2202_monk.eaf,0.093682,59037840,55308,0.0,0,0,0.206031,28408380,58530
2,P14_P7_2502_dizzy.eaf_&_P14_P7_2502_monk.eaf,0.064118,65597600,42060,0.0,0,0,0.211385,31501800,66590
3,P15_P13_2402_dizzy.eaf_&_P15_P13_2402_monk.eaf,0.047873,46654820,22335,0.013744,4875000,670,0.209721,31847090,66790
4,P16_P6_0903_dizzy.eaf_&_P16_P6_0903_monk.eaf,0.066983,52281680,35020,0.0,0,0,0.203548,28848270,58720
5,P18_P10_2102_dizzy.eaf_&_P18_P10_2102_monk.eaf,0.074556,58365170,43515,0.0,0,0,0.205708,28380000,58380
6,P18_P1_2102_dizzy.eaf_&_P18_P1_2102_monk.eaf,0.099832,58978920,58880,0.0,0,0,0.202144,28796240,58210
7,Total,0.53729,458922970.0,363613.0,0.02893,13534000.0,1985.0,1.445009,234617460.0,484570.0


**Database: IFADV**

,Pairs filenames,Overlap Percentage for Smiles_0 (%),Total Tier Duration for Smiles_0 (ms),Overlap Duration for Smiles_0 (ms),Overlap Percentage for Laughs_0 (%),Total Tier Duration for Laughs_0 (ms),Overlap Duration for Laughs_0 (ms),Overlap Percentage for Role (%),Total Tier Duration for Role (ms),Overlap Duration for Role (ms)
0,DVA1A.eaf_&_DVB1B.eaf,0.045931,116892370,53690,0.0,0,0,0.210914,56769460,119735
1,DVA2C.eaf_&_DVB2D.eaf,0.077648,118812180,92255,0.0,0,0,0.211373,57225905,120960
2,DVA3E.eaf_&_DVB3F.eaf,0.050904,39368380,20040,0.072581,3968000,2880,0.21124,28384730,59960
3,DVA4C.eaf_&_DVB4G.eaf,0.091891,52556640,48295,0.0,0,0,0.211381,28469870,60180
4,DVA5G.eaf_&_DVB5H.eaf,0.030879,62631960,19340,0.0,0,0,0.211317,30167940,63750
5,DVA6H.eaf_&_DVB6I.eaf,0.041784,60309530,25200,0.0,0,0,0.211348,29046930,61390
6,DVA7B.eaf_&_DVB7J.eaf,0.094881,58468280,55475,0.0,0,0,0.211258,28384730,59965
7,DVA8K.eaf_&_DVB8L.eaf,0.011289,56248960,6350,0.3721,3276000,12190,0.211346,28389460,60000
8,Total,0.445207,565288300.0,320645.0,0.444681,7244000.0,15070.0,1.690178,286839025.0,605940.0


**Database: NDC**

,Pairs filenames,Overlap Percentage for Smiles_0 (%),Total Tier Duration for Smiles_0 (ms),Overlap Duration for Smiles_0 (ms),Overlap Percentage for Laughs_0 (%),Total Tier Duration for Laughs_0 (ms),Overlap Duration for Laughs_0 (ms),Overlap Percentage for Role (%),Total Tier Duration for Role (ms),Overlap Duration for Role (ms)
0,13_1_A_M.eaf_&_13_1_B_F.eaf,0.023653,96169224,22747,0.02838,8950000,2540,0.211395,46321836,97922
1,13_2_A_M.eaf_&_13_2_B_F.eaf,0.045771,100317192,45916,0.0,0,0,0.207822,48319788,100419
2,13_4_A_M.eaf_&_13_4_B_F.eaf,0.029322,242785752,71189,0.015185,24426200,3709,0.211387,116942628,247201
3,14_1_A_M.eaf_&_14_1_B_F.eaf,0.007703,257883020,19865,0.0,0,0,0.175017,157967810,276470
4,14_2_A_M.eaf_&_14_2_B_F.eaf,0.018494,257197584,47567,0.03522,1590000,560,0.206498,123100615,254200
5,17_1_A_F.eaf_&_17_1_B_F.eaf,0.03408,86039894,29322,0.054522,5832500,3180,0.202255,40823211,82567
6,17_2_A_F.eaf_&_17_2_B_F.eaf,0.05788,131186362,75931,0.039584,11823000,4680,0.20708,63188543,130851
7,17_3_A_F.eaf_&_17_3_B_F.eaf,0.019256,121159160,23330,0.051334,12009500,6165,0.20575,58939584,121268
8,17_4_A_F.eaf_&_17_4_B_F.eaf,0.061495,200025544,123005,0.019928,13800000,2750,0.191567,96527002,184914
9,18_1_A_M.eaf_&_18_1_B_M.eaf,0.049057,171653600,84208,0.041907,3818000,1600,0.211374,83031839,175508


Now, we watch the pourcentage of overlap for each pair of file for list B to list A. (The total duration come from the files of list B)

In [17]:
lstA = {}
lstB = {}
overlapping_segments_dict = {}
for i, database in enumerate(databases_name):
    if database == databases_pairs[i].replace('_pairs', '').upper():
        databases_list = databases_pair_paths[databases_pairs[i]]
        dataset_dict = {}
        for i in range(0, len(databases_list), 2):
            filepath_A = databases_list[i+1]
            filepath_B = databases_list[i] 
            pair_file_A = os.path.basename(filepath_A)
            pair_file_B = os.path.basename(filepath_B)

            if pair_file_A and pair_file_B:
                pair_name = f"{pair_file_A}_&_{pair_file_B}"

            pair_dict = {}
            for tier in expressions + ["Role"]:
                lstA_tier = get_tier_from_file(filepath_A, tier)
                lstB_tier = get_tier_from_file(filepath_B, tier)

                if tier in lstA:
                    lstA[tier].extend(lstA_tier[tier])
                else:
                    lstA[tier] = lstA_tier[tier]

                if tier in lstB:
                    lstB[tier].extend(lstB_tier[tier])
                else:
                    lstB[tier] = lstB_tier[tier]

                overlapping_segments = get_overlapping_segments(lstA_tier[tier], lstB_tier[tier])
                pair_dict[tier] = {'Segments': overlapping_segments}

            dataset_dict[pair_name] = pair_dict

        overlapping_segments_dict[database] = dataset_dict

dataframes = {}  
for database, dataset_dict in overlapping_segments_dict.items():
    data = [] 
    for pair_name, pair_dict in dataset_dict.items():
        percentages = {}
        for tier, tier_dict in pair_dict.items():
            segments = tier_dict['Segments']
            if not segments:
                percentage = 0 
            else:
                overlapping_duration = sum(segment[1] - segment[0] for segment in segments)
                total_duration = sum(max(segment[1] for segment in segments) for segment in lstA[tier])
                percentage = (overlapping_duration / total_duration) * 100
            percentages[tier] = percentage
        
        data.append({'Pairs filenames': pair_name, **percentages})

    df = pd.DataFrame(data)
    df = df.applymap(lambda x: f"{x:.5f}%" if isinstance(x, float) else x)
    dataframes[database] = df

for database, df in dataframes.items():
    display(Markdown(f"**Dataset: {database}**"))
    display(df)

**Dataset: CCDB**

,Pairs filenames,Smiles_0,Laughs_0,Role
0,P12_P2_1402_monk.eaf_&_P12_P2_1402_dizzy.eaf,0.07663%,0.00449%,0.16499%
1,P12_P3_2202_monk.eaf_&_P12_P3_2202_dizzy.eaf,0.06408%,0.00000%,0.16593%
2,P14_P7_2502_monk.eaf_&_P14_P7_2502_dizzy.eaf,0.07119%,0.00000%,0.15856%
3,P15_P13_2402_monk.eaf_&_P15_P13_2402_dizzy.eaf,0.07726%,0.01745%,0.16649%
4,P16_P6_0903_monk.eaf_&_P16_P6_0903_dizzy.eaf,0.05869%,0.00000%,0.16138%
5,P18_P10_2102_monk.eaf_&_P18_P10_2102_dizzy.eaf,0.04361%,0.00000%,0.16415%
6,P18_P1_2102_monk.eaf_&_P18_P1_2102_dizzy.eaf,0.06728%,0.00000%,0.16226%


**Dataset: IFADV**

,Pairs filenames,Smiles_0,Laughs_0,Role
0,DVB1B.eaf_&_DVA1A.eaf,0.04678%,0.00000%,0.16863%
1,DVB2D.eaf_&_DVA2C.eaf,0.05893%,0.00000%,0.16861%
2,DVB3F.eaf_&_DVA3E.eaf,0.04120%,0.08297%,0.16683%
3,DVB4G.eaf_&_DVA4C.eaf,0.05992%,0.00000%,0.16858%
4,DVB5H.eaf_&_DVA5G.eaf,0.02762%,0.00000%,0.16861%
5,DVB6I.eaf_&_DVA6H.eaf,0.02876%,0.00000%,0.16863%
6,DVB7J.eaf_&_DVA7B.eaf,0.07929%,0.00000%,0.16852%
7,DVB8L.eaf_&_DVA8K.eaf,0.02497%,0.08874%,0.16859%


**Dataset: NDC**

,Pairs filenames,Smiles_0,Laughs_0,Role
0,13_1_B_F.eaf_&_13_1_A_M.eaf,0.03299%,0.03391%,0.16857%
1,13_2_B_F.eaf_&_13_2_A_M.eaf,0.05442%,0.00000%,0.16863%
2,13_4_B_F.eaf_&_13_4_A_M.eaf,0.03428%,0.01910%,0.16861%
3,14_1_B_F.eaf_&_14_1_A_M.eaf,0.00642%,0.00000%,0.16823%
4,14_2_B_F.eaf_&_14_2_A_M.eaf,0.01983%,0.04654%,0.16712%
5,17_1_B_F.eaf_&_17_1_A_F.eaf,0.02357%,0.03058%,0.16084%
6,17_2_B_F.eaf_&_17_2_A_F.eaf,0.05703%,0.04146%,0.16532%
7,17_3_B_F.eaf_&_17_3_A_F.eaf,0.03353%,0.01762%,0.16863%
8,17_4_B_F.eaf_&_17_4_A_F.eaf,0.06064%,0.01768%,0.16863%
9,18_1_B_M.eaf_&_18_1_A_M.eaf,0.04412%,0.02172%,0.16414%


For a more datailed analysis, we show here the duration of overlap for each pair of file for list B to list A, the total duration of the tier of list B and the pourcentage of overlap between B and A.

In [16]:
lstA = {}
lstB = {}
overlapping_segments_dict = {}
for i, database in enumerate(databases_name):
    if database == databases_pairs[i].replace('_pairs', '').upper():
        databases_list = databases_pair_paths[databases_pairs[i]]
        dataset_dict = {}
        for i in range(0, len(databases_list), 2):
            filepath_A = databases_list[i+1]
            filepath_B = databases_list[i]
            pair_file_A = os.path.basename(filepath_A)
            pair_file_B = os.path.basename(filepath_B)

            if pair_file_A and pair_file_B:
                pair_name = f"{pair_file_A}_&_{pair_file_B}"

            pair_dict = {}
            for tier in expressions + ["Role"]:
                lstA_tier = get_tier_from_file(filepath_A, tier)
                lstB_tier = get_tier_from_file(filepath_B, tier)

                if tier in lstA:
                    lstA[tier].extend(lstA_tier[tier])
                else:
                    lstA[tier] = lstA_tier[tier]

                if tier in lstB:
                    lstB[tier].extend(lstB_tier[tier])
                else:
                    lstB[tier] = lstB_tier[tier]

                overlapping_segments = get_overlapping_segments(lstA_tier[tier], lstB_tier[tier])
                pair_dict[tier] = {'Segments': overlapping_segments}

            dataset_dict[pair_name] = pair_dict

        overlapping_segments_dict[database] = dataset_dict

dataframes = {}
for database, dataset_dict in overlapping_segments_dict.items():
    tiers = expressions + ["Role"]
    data = {tier: [] for tier in tiers}
    for pair_name, pair_dict in dataset_dict.items():
        for tier in tiers:
            segments = pair_dict[tier]['Segments']
            if not segments:
                overlap_duration = 0
                total_duration = 0
                percentage = 0
            else:
                overlap_duration = sum(segment[1] - segment[0] for segment in segments)
                total_duration = sum(max(segment[1] for segment in segments) for segment in lstA[tier])
                percentage = (overlap_duration / total_duration) * 100
            data[tier].append({
                'Pairs filenames': pair_name,
                f'Overlap Percentage for {tier} (%)': percentage,
                f'Total Tier Duration for {tier} (ms)': total_duration,
                f'Overlap Duration for {tier} (ms)': overlap_duration
            })

    dfs = []
    for tier in tiers:
        df = pd.DataFrame(data[tier])
        dfs.append(df)

    df_merged = pd.concat(dfs, axis=1)
    df_merged = df_merged.loc[:, ~df_merged.columns.duplicated()]

    # Calculer le total de chaque colonne
    df_total = df_merged.sum(numeric_only=True)
    df_total['Pairs filenames'] = 'Total'

    # Ajouter la ligne "Total" à la fin du DataFrame
    df_merged = pd.concat([df_merged, pd.DataFrame(df_total).T], ignore_index=True)

    dataframes[database] = df_merged

for database, df in dataframes.items():
    display(Markdown(f"**Database: {database}**"))
    display(df)

**Database: CCDB**

,Pairs filenames,Overlap Percentage for Smiles_0 (%),Total Tier Duration for Smiles_0 (ms),Overlap Duration for Smiles_0 (ms),Overlap Percentage for Laughs_0 (%),Total Tier Duration for Laughs_0 (ms),Overlap Duration for Laughs_0 (ms),Overlap Percentage for Role (%),Total Tier Duration for Role (ms),Overlap Duration for Role (ms)
0,P12_P2_1402_monk.eaf_&_P12_P2_1402_dizzy.eaf,0.076634,149978400,114935,0.004485,11927340,535,0.164994,71426850,117850
1,P12_P3_2202_monk.eaf_&_P12_P3_2202_dizzy.eaf,0.064081,80421120,51535,0.0,0,0,0.165933,48127880,79860
2,P14_P7_2502_monk.eaf_&_P14_P7_2502_dizzy.eaf,0.071193,82542720,58765,0.0,0,0,0.158563,39221020,62190
3,P15_P13_2402_monk.eaf_&_P15_P13_2402_dizzy.eaf,0.077258,60702720,46898,0.017447,6877920,1200,0.166485,35832025,59655
4,P16_P6_0903_monk.eaf_&_P16_P6_0903_dizzy.eaf,0.058694,66156480,38830,0.0,0,0,0.161382,35574070,57410
5,P18_P10_2102_monk.eaf_&_P18_P10_2102_dizzy.eaf,0.043615,73931520,32245,0.0,0,0,0.164153,35704530,58610
6,P18_P1_2102_monk.eaf_&_P18_P1_2102_dizzy.eaf,0.067279,74904960,50395,0.0,0,0,0.162257,35597790,57760
7,Total,0.458755,588637920.0,393603.0,0.021933,18805260.0,1735.0,1.143767,301484165.0,493335.0


**Database: IFADV**

,Pairs filenames,Overlap Percentage for Smiles_0 (%),Total Tier Duration for Smiles_0 (ms),Overlap Duration for Smiles_0 (ms),Overlap Percentage for Laughs_0 (%),Total Tier Duration for Laughs_0 (ms),Overlap Duration for Laughs_0 (ms),Overlap Percentage for Role (%),Total Tier Duration for Role (ms),Overlap Duration for Role (ms)
0,DVB1B.eaf_&_DVA1A.eaf,0.046784,149784960,70075,0.0,0,0,0.168634,71171860,120020
1,DVB2D.eaf_&_DVA2C.eaf,0.05893,149760000,88253,0.0,0,0,0.168613,71174825,120010
2,DVB3F.eaf_&_DVA3E.eaf,0.0412,51941760,21400,0.082968,5435820,4510,0.166833,50526565,84295
3,DVB4G.eaf_&_DVA4C.eaf,0.059921,63516960,38060,0.0,0,0,0.168578,35864640,60460
4,DVB5H.eaf_&_DVA5G.eaf,0.027622,85475520,23610,0.0,0,0,0.168609,40685730,68600
5,DVB6I.eaf_&_DVA6H.eaf,0.028761,76702080,22060,0.0,0,0,0.168634,38331520,64640
6,DVB7J.eaf_&_DVA7B.eaf,0.079289,83403840,66130,0.0,0,0,0.168517,40163890,67683
7,DVB8L.eaf_&_DVA8K.eaf,0.024966,71897280,17950,0.088742,4558140,4045,0.168593,36226370,61075
8,Total,0.367472,732482400.0,347538.0,0.171711,9993960.0,8555.0,1.347012,384145400.0,646783.0


**Database: NDC**

,Pairs filenames,Overlap Percentage for Smiles_0 (%),Total Tier Duration for Smiles_0 (ms),Overlap Duration for Smiles_0 (ms),Overlap Percentage for Laughs_0 (%),Total Tier Duration for Laughs_0 (ms),Overlap Duration for Laughs_0 (ms),Overlap Percentage for Role (%),Total Tier Duration for Role (ms),Overlap Duration for Role (ms)
0,13_1_B_F.eaf_&_13_1_A_M.eaf,0.032994,122479968,40411,0.033915,12428280,4215,0.168565,58191090,98090
1,13_2_B_F.eaf_&_13_2_A_M.eaf,0.05442,126931584,69076,0.0,0,0,0.168626,60312844,101703
2,13_4_B_F.eaf_&_13_4_A_M.eaf,0.034277,308651616,105797,0.019098,33641640,6425,0.168614,146658981,247287
3,14_1_B_F.eaf_&_14_1_A_M.eaf,0.006421,325815360,20920,0.0,0,0,0.168226,198507936,333942
4,14_2_B_F.eaf_&_14_2_A_M.eaf,0.019832,327849600,65020,0.046545,2191440,1020,0.167119,155781100,260340
5,17_1_B_F.eaf_&_17_1_A_F.eaf,0.023572,109669248,25851,0.030579,8159250,2495,0.160835,51033580,82080
6,17_2_B_F.eaf_&_17_2_A_F.eaf,0.057031,166770240,95110,0.041465,16519980,6850,0.165315,79247334,131008
7,17_3_B_F.eaf_&_17_3_A_F.eaf,0.033529,154252800,51720,0.017617,16546200,2915,0.168634,74181928,125096
8,17_4_B_F.eaf_&_17_4_A_F.eaf,0.060644,253712160,153860,0.017677,19121280,3380,0.168626,121001650,204040
9,18_1_B_M.eaf_&_18_1_A_M.eaf,0.04412,218548512,96424,0.021716,5249520,1140,0.164142,103845567,170454


As we can see, the result of overlapping is not the same for the two lists. We can see that the pourcentage of overlap: 